In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Normalization, TextVectorization
from tensorflow.keras.layers.experimental.preprocessing import \
    TextVectorization

model = keras.models.load_model('model')

In [24]:
# Load the trained model
model = keras.models.load_model('model')

# Load the test dataframe
df_test = pd.read_parquet('val.parquet')

# Get the textual and numerical data from the test dataframe
textual_data = df_test['Textual'].to_numpy()
numerical_data = df_test['Numerical'].to_numpy()

vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200, output_mode='int')

# Fit the vectorizer on the textual data
vectorizer.adapt(textual_data)

normalizer = Normalization(axis=None)

# Vectorize the textual data using the same vectorizer used during training
vectorized_textual_data = vectorizer(textual_data)

# Reshape the textual data to a 1D tensor of strings and cast it to float32
textual_data_1d = tf.cast(tf.reshape(vectorized_textual_data, (-1,)), tf.float32)
textual_data_1d = tf.reshape(textual_data_1d, (-1,))

# Normalize the numerical data using the same normalizer used during training and cast it to float32
normalized_numerical_data = tf.cast(normalizer(numerical_data), tf.float32)

# Concatenate the textual and numerical data along the second axis
input_data = tf.concat([textual_data_1d[:, tf.newaxis], normalized_numerical_data], axis=1)

# Get predicted probabilities for each entry in the test dataset
pred_probs = model.predict(input_data)


InvalidArgumentError: {{function_node __wrapped__ConcatV2_N_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} ConcatOp : Ranks of all input tensors should match: shape[0] = [4000,1] vs. shape[1] = [20] [Op:ConcatV2] name: concat